[notion노트](https://www.notion.so/Minpy40-41-e29e82cef51e4d92b114372368860f32)

In [1]:
import pandas as pd

train_df = pd.read_csv('datas/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [2]:
train_df['label'].value_countsunts()

0    75173
1    74827
Name: label, dtype: int64

In [4]:
import re
train_df = train_df.fillna(' ')

train_df['document'] = train_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

In [5]:
test_df = pd.read_csv('datas/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x))

In [7]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    tokens_ko = twitter.v(text)
    return tokens_ko

c:\users\oaqqa\anaconda3\envs\nltk\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer,
                            ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

c:\users\oaqqa\anaconda3\envs\nltk\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [10]:
!pip install lightgbm

  Using cached lightgbm-3.1.1-py2.py3-none-win_amd64.whl (754 kB)


In [11]:
from lightgbm import LGBMClassifier
import time

start_time = time.time()
lgbm_clf = LGBMClassifier(n_estimators=400)
lgbm_clf.fit(tfidf_matrix_train, train_df['label'])
print('Fit time : ', time.time() - start_time)

Fit time :  176.84099507331848


In [12]:
from sklearn.metrics import accuracy_score

tfidf_matrix_test = tfidf_vect.transform(test_df['document'])
preds = lgbm_clf.predict(tfidf_matrix_test)

accuracy_score(test_df['label'], preds)

0.82958

In [13]:
test_df['document'][100]

'걸작은 몇안되고 졸작들만 넘쳐난다.'

In [15]:
lgbm_clf.predict(tfidf_vect.transform(test_df['document'])[100])

array([0], dtype=int64)